# Step 1. Feature extraction

In [1]:
import sys
import os
import pickle
import tensorflow as tf
from tensorflow.contrib.slim.python.slim.nets import alexnet, vgg, inception_v3, resnet_v2

In [4]:
alex_size = alexnet.alexnet_v2.default_image_size
vgg_size = vgg.vgg_16.default_image_size
inc_size = inception_v3.inception_v3.default_image_size
res_size = resnet_v2.resnet_v2.default_image_size
print('alexnet:%d, vgg:%d, inception:%d, resnet:%d' %(alex_size, vgg_size, inc_size, res_size))

alexnet:224, vgg:224, inception:299, resnet:224


In [3]:
def feature_extraction(class_count, final_tensor_name, bottleneck_tensor):
    with tf.name_scope('input'):
        bottleneck_input = tf.placeholder_with_default(
            bottleneck_tensor, shape=[None, BOTTLENECK_TENSOR_SIZE],
            name='BottleneckInputPlaceholder')

    ground_truth_input = tf.placeholder(tf.float32,
                                        [None, class_count],
                                        name='GroundTruthInput')
    
    layer_name = 'final_training_ops'
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            layer_weights = tf.Variable(tf.truncated_normal([BOTTLENECK_TENSOR_SIZE, class_count], stddev=0.001), name='final_weights')
            variable_summaries(layer_weights)
        with tf.name_scope('biases'):
            layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
            variable_summaries(layer_biases)
        with tf.name_scope('Wx_plus_b'):
            logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
            tf.summary.histogram('pre_activations', logits)
    
    final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
    tf.summary.histogram('activations', final_tensor)

    with tf.name_scope('cross_entropy'):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
            labels=ground_truth_input, logits=logits)
        with tf.name_scope('total'):
            cross_entropy_mean = tf.reduce_mean(cross_entropy)
    tf.summary.scalar('cross_entropy', cross_entropy_mean)

    with tf.name_scope('train'):
        train_step = tf.train.GradientDescentOptimizer(FLAGS.learning_rate).minimize(
            cross_entropy_mean)
    
    return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
            final_tensor)

In [ ]:
# returns the second last layer of alexnet
fc7 = alexnet.alexnet_v2
fc7 = tf.stop_gradient(fc7)
# shape = (fc7.get_shape().as_list()[-1], nb_classes)
# fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
# fc8b = tf.Variable(tf.zeros(nb_classes))
# logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

# cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
# loss_op = tf.reduce_mean(cross_entropy)
# opt = tf.train.AdamOptimizer()
# train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
# init_op = tf.global_variables_initializer()

# preds = tf.arg_max(logits, 1)
# accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

In [ ]:
nb_classes = 3
epochs = 10
batch_size = 20

features = tf.placeholder(tf.float32, (None, 500, 375, 3))
labels = tf.placeholder(tf.int64, None)

alexnet
# fc7 = alexnet(features, feature_extract=True)
# fc7 = tf.stop_gradient(fc7)
# shape = (fc7.get_shape().as_list()[-1], nb_classes)
# fc8W = tf.Variable(tf.truncated_normal(shape, stddev=1e-2))
# fc8b = tf.Variable(tf.zeros(nb_classes))
# logits = tf.nn.xw_plus_b(fc7, fc8W, fc8b)

# cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels)
# loss_op = tf.reduce_mean(cross_entropy)
# opt = tf.train.AdamOptimizer()
# train_op = opt.minimize(loss_op, var_list=[fc8W, fc8b])
# init_op = tf.global_variables_initializer()

# preds = tf.arg_max(logits, 1)
# accuracy_op = tf.reduce_mean(tf.cast(tf.equal(preds, labels), tf.float32))

Credit: http://www.cs.toronto.edu/~guerzhoy/tf_alexnet/myalexnet_forward.py